In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import sklearn
from sklearn.model_selection import train_test_split

# Import skvideo. A library that provides easy access to common as well as state-of-the-art video processing routines.

! sudo pip install scikit-video

import skvideo.io

In [ ]:
#Mount google drive
from google.colab import drive
#For authentication 
drive.mount('/gdrive')
%cd /gdrive

In [ ]:
videodata = skvideo.io.vread("/gdrive/MyDrive/pla(i)nt/Video Folders of pla(i)nt/Soil_Temp_Lvl_1_18_24_20_10.mp4")
print(videodata.shape)

In [ ]:
"""
Isolate the first channel band which corresponds to the first level of Soil Temperature.

Soil Temperature Level 1 corresponds to the first layer below Earth's surface level. The first 7cm below surface level translate to Level 1 soil. 
"""
videodata = videodata[:,:,:,:1]
print(videodata.shape)

In [5]:
# Hard-set season variable to assess seasonality patterns per example

def pre_data(num_frames_per_example):

    videodata = skvideo.io.vread("/gdrive/MyDrive/pla(i)nt/Video Folders of pla(i)nt/Soil_Temp_Lvl_1_18_24_20_10.mp4")
    videodata = videodata[:,:,:,:1]

    data = []
    pre_exmaples = []

    for i in range (videodata.shape[0] - num_frames_per_example):
        
        pre_examples = videodata[i:i + num_frames_per_example, :, :, :]
        data.append(pre_examples)
    
    dataset = np.array(data)
    # print("We have", dataset.shape[0],"examples")
    # print("Each example has", dataset.shape[1],"frames")
    # print("Each frame is", dataset.shape[2],"by",dataset.shape[3])

    return dataset

In [6]:

"""
In this case we have 480 frames. Each frame is an image with spatial dims of (12 X 32).

We set a season like (Autumn, Winter...) as the base for our forecast. So we plot 3 consecutive frames each time in order to assess if there are any distored frames from 
our library translation
"""

def data_vis(num_examples):
    
    for i in range (num_examples):

        random_data_choice = np.random.choice(a = pre_data(3).shape[0], size = 1)[0]

        for j in range(pre_data(3).shape[1]):

            image = plt.imshow(np.squeeze(pre_data(3)[random_data_choice][j]), cmap = "hot", aspect = "equal")
            plt.title("Soil temperature in frame "  f"{j + 1} " f"of example {random_data_choice}" )
            plt.colorbar()
            plt.show()

    return random_data_choice

In [7]:
def month_vis(number_of_month):
    
    videodata = skvideo.io.vread("/gdrive/MyDrive/pla(i)nt/Video Folders of pla(i)nt/Soil_Temp_Lvl_1_18_24_20_10.mp4")
    videodata = videodata[:,:,:,:1]

    videodata = videodata[number_of_month::12]

    month_dict = {1:'January', 2:'February', 3:'March', 4:'April', 5:'May', 6:'June', 7:'July', 
                  8:'August', 9:'September', 10:'October', 11:'November', 12:'December'}

    for i in range(videodata.shape[0]):

        image = plt.imshow(np.squeeze(videodata[i]), cmap = "hot", aspect = "equal")
        plt.title("Soil temperature of Crete in " f"{month_dict[number_of_month]}"  f"{1981 + i}" )
        plt.colorbar()
        plt.show()
    
    return month_dict[number_of_month]

In [ ]:
train_data, test_data = sklearn.model_selection.train_test_split(pre_data(3), test_size = 0.3, train_size = 0.7)
print("Our training data have shape", train_data.shape)
print("Our testing data have shape", test_data.shape)

In [9]:
train_data = train_data / 255.0
test_data = test_data / 255.0

In [ ]:
def assign_frames(data):
  x = data[:, 0 : data.shape[1] - 1, :, :, :]
  y = data[:, 1 : data.shape[1], :, :, :]
  return x, y

x_train, y_train = assign_frames(train_data)

print("Training Dataset Shape is: " + str(x_train.shape) + ", " + str(y_train.shape))

x_test, y_test = assign_frames(test_data)
print("Testing Dataset Shape is: " + str(x_test.shape) + ", " + str(y_test.shape))

In [11]:
def pix_evol(row_pix, col_pix):

    videodata = skvideo.io.vread("/gdrive/MyDrive/pla(i)nt/Video Folders of pla(i)nt/Soil_Temp_Lvl_1_18_24_20_10.mp4")/255
    videodata = videodata[:,:,:,:1]
    
    pixel_evol = np.swapaxes(videodata,0, 2)
    
    if ((row_pix == 6) and (col_pix == 23)):

        ag_nik_pix_evol = pixel_evol[col_pix][row_pix][:]
        plt.plot(ag_nik_pix_evol)
        plt.xlabel("Number of frames")
        plt.ylabel("Pixel Value")
        plt.title("Time evolution of Soil Temperature at Ag. Nikolaos")
    
    elif ((row_pix == 5) and (col_pix == 17)):

        her_pix_evol = pixel_evol[col_pix][row_pix][:]
        plt.plot(her_pix_evol)
        plt.xlabel("Number of frames")
        plt.ylabel("Pixel Value")
        plt.title("Time evolution of Soil Temperature at Heraklion")
        
    elif ((row_pix == 5) and (col_pix == 10)):

        rethymno_pix_evol = pixel_evol[col_pix][row_pix][:]
        plt.plot(rethymno_pix_evol)
        plt.xlabel("Number of frames")
        plt.ylabel("Pixel Value")
        plt.title("Time evolution of Soil Temperature at Rethymno")

    elif((row_pix == 2) and (col_pix == 6)):

        chania_pix_evol = pixel_evol[col_pix][row_pix][:]
        plt.plot(chania_pix_evol)
        plt.xlabel("Number of frames")
        plt.ylabel("Pixel Value")
        plt.title("Time evolution of Soil Temperature at Chania")

    else:

        random_pix = pixel_evol[col_pix][row_pix][:]
        plt.plot(random_pix)
        plt.xlabel("Number of frames")
        plt.ylabel("Pixel Value")
        plt.title("Time evolution of Soil Temperature at random area in Crete")

    plt.show()

In [ ]:
data_panda = pd.read_csv("/gdrive/MyDrive/pla(i)nt/Csv Folders of pla(i)nt/Heraklion pandas.csv")
data_panda.plot.line(x='Time',y='soil_temperature_level_1',color='red',legend=None)
plt.xticks(rotation=45)
plt.ylabel('Soil Temperature (K)')
plt.xlabel('Time (Years)')
plt.title('Soil Temperature at Heraklion')
plt.show()

In [13]:
# Construct the input layer with no definite frame size.
inp = layers.Input(shape=(None, *x_train.shape[2:]))

# We will construct 3 `ConvLSTM2D` layers with batch normalization,
# followed by a `Conv3D` layer for the spatiotemporal outputs.
x = layers.ConvLSTM2D(
    filters=64,
    kernel_size=(5, 5),
    padding="same",
    return_sequences=True,
    activation="relu",
)(inp)
x = layers.BatchNormalization()(x)
x = layers.ConvLSTM2D(
    filters=64,
    kernel_size=(3, 3),
    padding="same",
    return_sequences=True,
    activation="relu",
)(x)
x = layers.BatchNormalization()(x)
x = layers.ConvLSTM2D(
    filters=64,
    kernel_size=(1, 1),
    padding="same",
    return_sequences=True,
    activation="relu",
)(x)
x = layers.Conv3D(
    filters=1, kernel_size=(3, 3, 3), activation="sigmoid", padding="same"
)(x)

In [14]:
# Next, we will build the complete model and compile it.
model = keras.models.Model(inp, x)
model.compile(
    loss=keras.losses.mean_squared_error, optimizer=keras.optimizers.Adam(0.0001))

In [ ]:
# Define some callbacks to improve training.
#early_stopping = keras.callbacks.EarlyStopping(monitor="accuracy", patience=10)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor="loss", patience = 2)

# Define modifiable training hyperparameters.
epochs = 10
batch_size = 10

# Fit the model to the training data.
history = model.fit(x_train, y_train, epochs = epochs, batch_size = batch_size, callbacks = [reduce_lr])

In [ ]:
plt.xlabel('Epochs')
plt.ylabel("Loss Magnitude")
plt.title('Training Loss')
plt.plot(history.history['loss'])
plt.show()